challenge 4 (crimen en la libreria SQL):    

In [ ]:
# ------- almacen de datos ------
authors_books = []

In [2]:
# ------- funciones importantes -------

import requests
import time


# ----- funcion para hacer request a los url  ------
def hacer_requests(URL, params=None, reintentos_posibles = 5, tiempo_espera = 1):
    time.sleep(tiempo_espera)
    for intentos in range(reintentos_posibles):
        try:
            respuesta = requests.get(URL, params=params)
            return respuesta
        except:
            print(f"[error de conexion] : [intento {intentos + 1}] : [tiempo de espera : {tiempo_espera}]")
            time.sleep(tiempo_espera)
            tiempo_espera *= 2

In [ ]:
# ------- Escraper en si -------

from bs4 import BeautifulSoup
from urllib import parse

# ----- URL a usar ------
url_booksToScrape = "https://books.toscrape.com/index.html"
url_api_googleBooks = ""

# ----- hacer conexion y soup ------
respuesta = hacer_requests(url_booksToScrape)
soup = BeautifulSoup(respuesta.content, "lxml")

# ------ recoger todas las categorias -----
side_categories_tag = soup.find("div", class_="side_categories")
nav_list_tag = side_categories_tag.find("ul", class_="nav nav-list")
ubication_categories_tag = nav_list_tag.find("ul")
list_all_categories_tag = ubication_categories_tag.find_all("li")

for elemento in list_all_categories_tag:
    
    # ------- nombre de la categoria --------
    category_name = elemento.a.get_text(strip=True)
    # ----------------------------------------

    category_reference_url = elemento.a["href"]

    new_url = parse.urljoin(url_booksToScrape, category_reference_url)
    response = hacer_requests(new_url)
    category_eleccion_soup = BeautifulSoup(response.content, "lxml")

    # --------- encontrar todos los libros de la categoria --------
    alert_alert_warning_tag = category_eleccion_soup.find("div", class_="alert alert-warning")
    
    list_of_books_tags = alert_alert_warning_tag.find_all("article", class_="product_pod")

    for book in list_of_books_tags:
        print(book.prettify())
        break
    break





    


https://books.toscrape.com/catalogue/category/books/travel_2/index.html
